In [1]:
# This will connect to a postgres database using the public IP address of the VM
# The bronze tables to create the silver tables

import os 
import sqlalchemy
import pandas as pd
from shapely.geometry import Point, Polygon
import datetime
import psycopg2
import pandas as pd
import geopandas as gpd


## taxi_covid

information about the destinations of taxis, mapped to the destination zip code with information about the weekly covid rates

In [42]:

user = 'postgres'
password = '432'
ip = '34.134.248.227'
port = '5432'



In [3]:
def get_zip(lat, long):
    '''
    Gets the zip code comparing the latitude and longitude to the zips.geojson file
    
    '''
    point = Point(long, lat)
    for i in range(len(zips)):
        if point.within(zips['geometry'][i]):
            return zips['zip'][i]
        else:
            continue


In [43]:
# connect to database
db_name = 'bronze'
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{ip}/{db_name}")

# select latitude and longitude from the table and run
sql = "SELECT trip_start_timestamp as ride_date, dropoff_centroid_latitude as lat, dropoff_centroid_longitude as long, trip_id FROM taxi_trips;"
df_taxi = pd.read_sql(sql, engine)
sql = "SELECT trip_start_timestamp as ride_date, dropoff_centroid_latitude as lat, dropoff_centroid_longitude as long, trip_id  FROM tnp_trips;"
df_tnp = pd.read_sql(sql,engine)

# load zips.geojson file with geopandas
zips = gpd.read_file('zips.geojson')



In [5]:
# Takes the latitude and longitude from df and create a zip code column
# Uses reverse_geocoder to find the zip code for each latitude and longitude using the zips.geojson file

zip_list = [get_zip(lat_long[1]['lat'],lat_long[1]['long']) for lat_long in df_taxi[['long','lat']].iterrows()]
df_taxi['zip'] = zip_list
df_taxi['ride_type'] = 'taxi'

zip_list = [get_zip(lat_long[1]['lat'],lat_long[1]['long']) for lat_long in df_tnp[['long','lat']].iterrows()]
df_tnp['zip'] = zip_list
df_tnp['ride_type'] = 'rideshare'

# append the datasets
df_ride = df_taxi.append(df_tnp, ignore_index=True, sort='ride_date')

# get the week number, so we can join on the weekly covid info
df_ride['week_number'] = [int(item.strftime("%W")) for item in df_ride['ride_date']]
df_ride['year'] = [item.strftime('%Y') for item in df_ride['ride_date']]

# df.to_csv('./src/python/df.csv', encoding='utf-8', index=False)

In [6]:
# now for the covid data
sql_query = '''SELECT zip_code as zip, week_number, cases_weekly, week_start,
                case_rate_cumulative, deaths_weekly, death_rate_weekly 
                from covid_19_zip;'''
df_covid_zip = pd.read_sql(sql_query, conn)
df_covid_zip.rename(columns={'cases_weekly':'covid_rate'}, inplace=True)
df_covid_zip.rename(columns={'death_rate_weekly':'death_rate'}, inplace=True)


df_covid_zip['year'] = [item.strftime('%Y') for item in df_covid_zip['week_start']]

In [7]:
# and to join on the zip and week_number
df_taxi_covid = pd.merge(df_ride, df_covid_zip, how= 'left', on=['year','week_number','zip'])

In [40]:
# close out the bronze connection
# insert into the silver database :)
user = 'postgres'
password = '432'
ip = '34.134.248.227'
db_name = 'silver'
port = '5432'

# conn = psycopg2.connect(f'host={ip} dbname={db_name} user={user} password={password} port={port}')
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{ip}/{db_name}")

num_inserts = df_taxi_covid[['ride_type','ride_date','zip','covid_rate','trip_id','death_rate']].\
    to_sql(name = 'taxi_covid', con=engine, if_exists='append', index=False, chunksize=1000)


## midway_taxi_covid

mostly the same as above, but looking to see if the destination is midway

In [67]:
midway_zip = get_zip(41.786, -87.7525)
ohare_zip = get_zip(41.978611,-87.904722)

mdw_rows = df_taxi_covid['zip'].eq(midway_zip)
midway_taxi_covid = df_taxi_covid.loc[mdw_rows,['ride_date','ride_type','zip','covid_rate']]

ohare_rows = df_taxi_covid['zip'].eq(ohare_zip)
ohare_taxi_covid = df_taxi_covid.loc[ohare_rows,['ride_date','ride_type','zip','covid_rate']]

mdw_inserts = midway_taxi_covid.to_sql('midway_taxi_covid', con=engine,\
                                       if_exists='append', index=False, chunksize=1000)
ord_inserts = ohare_taxi_covid.to_sql('ohare_taxi_covid', con=engine,\
                                      if_exists='append', index=False, chunksize=1000)


None to midway, None to ohare


In [66]:
user = 'postgres'
password = '432'
ip = '34.134.248.227'
db_name = 'silver'
port = '5432'
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{ip}/{db_name}")